In [1]:
import sqlite3
import random
from faker import Faker
from datetime import datetime

fake = Faker()

In [2]:
# Connect to (or create) the SQLite database file.
conn = sqlite3.connect("shop.db")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS addresses (
    address_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    street TEXT,
    city TEXT,
    state TEXT,
    zip_code TEXT,
    country TEXT,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT UNIQUE NOT NULL,
    description TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    category_id INTEGER,
    description TEXT,
    price REAL NOT NULL,
    stock_quantity INTEGER DEFAULT 0,
    FOREIGN KEY(category_id) REFERENCES categories(category_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    total_amount REAL NOT NULL,
    shipping_address_id INTEGER,
    FOREIGN KEY(user_id) REFERENCES users(user_id),
    FOREIGN KEY(shipping_address_id) REFERENCES addresses(address_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS order_items (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER NOT NULL,
    payment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    amount REAL NOT NULL,
    payment_method TEXT,
    FOREIGN KEY(order_id) REFERENCES orders(order_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    review_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product_id INTEGER,
    rating INTEGER CHECK (rating >= 1 AND rating <= 5),
    comment TEXT,
    review_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(user_id) REFERENCES users(user_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
);
""")

conn.commit()

In [3]:
#####################################
# Populate Users and Addresses
#####################################
num_users = 30
users_data = []
addresses_data = []
for _ in range(num_users):
    username = fake.user_name()
    email = fake.email()
    password_hash = fake.sha256()
    users_data.append((username, email, password_hash))
cursor.executemany("INSERT INTO users (username, email, password_hash) VALUES (?, ?, ?);", users_data)
conn.commit()

# Retrieve user_ids for address generation.
cursor.execute("SELECT user_id FROM users;")
user_ids = [row[0] for row in cursor.fetchall()]

# For simplicity, create one address per user.
for user_id in user_ids:
    street = fake.street_address()
    city = fake.city()
    state = fake.state_abbr()
    zip_code = fake.zipcode()
    country = fake.country()
    addresses_data.append((user_id, street, city, state, zip_code, country))
cursor.executemany(
    "INSERT INTO addresses (user_id, street, city, state, zip_code, country) VALUES (?, ?, ?, ?, ?, ?);",
    addresses_data,
)
conn.commit()

#####################################
# Populate Categories
#####################################
categories_list = [
    "Electronics", "Books", "Clothing", "Home", "Toys",
    "Sports", "Beauty", "Grocery", "Automotive", "Office"
]
categories_data = []
for cat in categories_list:
    description = fake.sentence(nb_words=10)
    categories_data.append((cat, description))
cursor.executemany("INSERT INTO categories (category_name, description) VALUES (?, ?);", categories_data)
conn.commit()

# Get category ids.
cursor.execute("SELECT category_id FROM categories;")
category_ids = [row[0] for row in cursor.fetchall()]

#####################################
# Populate Products
#####################################
num_products = 100
products_data = []
# We'll also store product prices in a list for later reference.
products_list = []
for i in range(num_products):
    product_name = fake.catch_phrase()
    category_id = random.choice(category_ids)
    description = fake.text(max_nb_chars=100)
    price = round(random.uniform(5, 2000), 2)
    stock_quantity = random.randint(0, 500)
    products_data.append((product_name, category_id, description, price, stock_quantity))
cursor.executemany(
    "INSERT INTO products (product_name, category_id, description, price, stock_quantity) VALUES (?, ?, ?, ?, ?);",
    products_data,
)
conn.commit()
# Assuming product_id is sequential (1-indexed).
for i, prod in enumerate(products_data, start=1):
    products_list.append({"product_id": i, "price": prod[3]})

#####################################
# Populate Orders and Order Items (with Payment Details)
#####################################
num_orders = 200
orders_data = []
order_items_data = []
order_totals = {}  # To store computed total for each order.

# Insert orders with a temporary total_amount of 0.
for _ in range(num_orders):
    user_id = random.choice(user_ids)
    order_date = fake.date_time_between(start_date="-1y", end_date="now")
    # For simplicity, use the user's address (addresses were inserted in order).
    shipping_address_id = user_id  
    orders_data.append((user_id, order_date, 0.0, shipping_address_id))
cursor.executemany(
    "INSERT INTO orders (user_id, order_date, total_amount, shipping_address_id) VALUES (?, ?, ?, ?);",
    orders_data,
)
conn.commit()

# Retrieve orders to generate order items.
cursor.execute("SELECT order_id, user_id, order_date, shipping_address_id FROM orders;")
orders_records = cursor.fetchall()

payment_methods = ["Credit Card", "PayPal", "Debit Card"]
for order in orders_records:
    order_id, user_id, order_date, shipping_address_id = order
    total = 0.0
    # Generate a single payment_date and payment_method for this order.
    payment_date = fake.date_time_between_dates(datetime_start=datetime.strptime(order_date, "%Y-%m-%d %H:%M:%S"), datetime_end=datetime.now())
    payment_method = random.choice(payment_methods)
    
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product = random.choice(products_list)
        product_id = product["product_id"]
        unit_price = product["price"]
        quantity = random.randint(1, 10)
        amount = unit_price * quantity
        total += amount
        # Note: Each order_item gets the same payment details as the parent order.
        order_items_data.append((order_id, product_id, quantity, payment_date, amount, payment_method))
    order_totals[order_id] = round(total, 2)

cursor.executemany(
    "INSERT INTO order_items (order_id, product_id, quantity, payment_date, amount, payment_method) VALUES (?, ?, ?, ?, ?, ?);",
    order_items_data,
)
conn.commit()

# Update each order with the computed total_amount.
for order_id, total in order_totals.items():
    cursor.execute("UPDATE orders SET total_amount = ? WHERE order_id = ?;", (total, order_id))
conn.commit()

#####################################
# Populate Reviews
#####################################
num_reviews = 300
reviews_data = []
for _ in range(num_reviews):
    user_id = random.choice(user_ids)
    product_id = random.randint(1, num_products)
    rating = random.randint(1, 5)
    comment = fake.sentence(nb_words=12)
    review_date = fake.date_time_between(start_date="-1y", end_date="now")
    reviews_data.append((user_id, product_id, rating, comment, review_date))
cursor.executemany(
    "INSERT INTO reviews (user_id, product_id, rating, comment, review_date) VALUES (?, ?, ?, ?, ?);",
    reviews_data,
)
conn.commit()

print("Database populated successfully with Faker data!")
conn.close()

Database populated successfully with Faker data!


C:\Users\jochua\AppData\Local\Temp\ipykernel_15912\1444854208.py:89: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(
C:\Users\jochua\AppData\Local\Temp\ipykernel_15912\1444854208.py:119: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(
C:\Users\jochua\AppData\Local\Temp\ipykernel_15912\1444854208.py:142: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(


In [4]:
def mask_card_number(card_number):
    # Masks all digits except the last 4.
    return "*" * (len(card_number) - 4) + card_number[-4:]

# Connect to (or create) the financial database file.
conn = sqlite3.connect("finance.db")
cursor = conn.cursor()

#####################################
# Create Tables for Financial Data
#####################################

# Financial Payments Table with a masked_card column.
cursor.execute("""
CREATE TABLE IF NOT EXISTS financial_payments (
    payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER,
    payment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    amount REAL NOT NULL,
    payment_method TEXT,  -- Abbreviated: CC, pp, DC
    masked_card TEXT,
    status TEXT
);
""")

# Cost Prices Table for tracking the cost price of products.
cursor.execute("""
CREATE TABLE IF NOT EXISTS cost_prices (
    cost_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    cost_price REAL NOT NULL,
    effective_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")
conn.commit()

In [5]:
#####################################
# Populate Financial Payments
#####################################

num_payments = 200
payments_data = []

# Map full payment method names to abbreviated forms.
payment_method_map = {
    "Credit Card": "CC",
    "PayPal": "pp",
    "Debit Card": "DC"
}
payment_methods_full = list(payment_method_map.keys())

for _ in range(num_payments):
    order_id = random.randint(1, 200)  # Assuming overlapping order IDs.
    payment_date = fake.date_time_between(start_date="-1y", end_date="now")
    amount = round(random.uniform(10, 2000), 2)
    full_method = random.choice(payment_methods_full)
    payment_method_abbr = payment_method_map[full_method]
    status = "Completed"
    
    # Generate a masked card number only for card-based payments.
    masked_card = None
    if full_method in ["Credit Card", "Debit Card"]:
        card_number = fake.credit_card_number(card_type=None)
        masked_card = mask_card_number(card_number)
    
    payments_data.append((order_id, payment_date, amount, payment_method_abbr, masked_card, status))

cursor.executemany("""
INSERT INTO financial_payments (order_id, payment_date, amount, payment_method, masked_card, status)
VALUES (?, ?, ?, ?, ?, ?);
""", payments_data)
conn.commit()

#####################################
# Populate Cost Prices
#####################################
# Assuming 100 products overlap with the ecommerce database.
num_products = 100
cost_prices_data = []

for product_id in range(1, num_products + 1):
    # Generate a selling price and then compute a cost price as 50%-90% of it.
    selling_price = random.uniform(20, 2000)
    cost_price = round(selling_price * random.uniform(0.5, 0.9), 2)
    effective_date = fake.date_time_between(start_date="-2y", end_date="now")
    cost_prices_data.append((product_id, cost_price, effective_date))

cursor.executemany("""
INSERT INTO cost_prices (product_id, cost_price, effective_date)
VALUES (?, ?, ?);
""", cost_prices_data)
conn.commit()

print("Financial database populated successfully with masked card numbers!")
conn.close()

Financial database populated successfully with masked card numbers!


C:\Users\jochua\AppData\Local\Temp\ipykernel_15912\3659411254.py:32: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany("""
C:\Users\jochua\AppData\Local\Temp\ipykernel_15912\3659411254.py:52: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany("""
